In [25]:
import tensorflow as tf
from tensorflow.keras.models import save_model
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Input, InputLayer, Conv2D, MaxPooling2D, Reshape, Flatten
from keras.models import load_model
from keras.datasets import mnist

1. Create Keras Model

In [10]:
def create_sequential_model():
    single_output_model = Sequential([
        InputLayer(input_shape=(28, 28)),
        Reshape((28,28,1),input_shape=(28,28,)),
        Conv2D(filters=6, kernel_size=5, strides=1, input_shape=(28, 28, 1), name="Conv2D_1"),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="MaxPooling2D_1"),
        Conv2D(filters=16, kernel_size=5, strides=1, name="Conv2D_2"),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="MaxPooling2D_2"),
        Flatten(),
        Dense(120, activation="relu", name="Dense_1"),
        Dense(84, activation="relu", name="Dense_2"),
        Dense(10, activation="softmax", name="Softmax")
    ])
    return single_output_model

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = create_sequential_model()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
MaxPooling2D_1 (MaxPooling2D (None, 12, 12, 6)         0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
MaxPooling2D_2 (MaxPooling2D (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
Dense_1 (Dense)              (None, 120)               30840     
__________

In [12]:
for layer in model.layers:
     print(layer.name)

reshape_2
Conv2D_1
MaxPooling2D_1
Conv2D_2
MaxPooling2D_2
flatten_2
Dense_1
Dense_2
Softmax


In [13]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 24s 399us/step - loss: 0.1943 - acc: 0.9403
Epoch 2/5
60000/60000 [==============================] - 24s 406us/step - loss: 0.0710 - acc: 0.9780
Epoch 3/5
60000/60000 [==============================] - 24s 402us/step - loss: 0.0504 - acc: 0.9839
Epoch 4/5
60000/60000 [==============================] - 24s 402us/step - loss: 0.0393 - acc: 0.9876
Epoch 5/5
60000/60000 [==============================] - 24s 403us/step - loss: 0.0317 - acc: 0.9902


In [14]:
input_sample = np.ndarray(shape=(28,28), buffer=np.random.rand(28,28))
input_sample = np.expand_dims(input_sample, axis=0)
print(model.predict(input_sample))

[[0.08669154 0.7764022  0.0106282  0.00502056 0.05059609 0.01246533
  0.00419575 0.00660756 0.0123862  0.03500653]]


2. Generate our encapsulated model which contains multiple intermediate outputs:

We can try to predict an input sample with the encapsulated model:
The output is a list which contains all output data of the layers we selected.
The last output is the same as the one we predicted from the original model

In [15]:
output_layer_names = [
    "Conv2D_1", "MaxPooling2D_1", "Conv2D_2", "MaxPooling2D_2",
    "Dense_1", "Dense_2", "Softmax"
]

def generate_encapsulate_model_with_output_layer_names(model, output_layer_names):
    enc_model = Model(
        inputs=model.input,
        outputs=list(map(lambda oln: model.get_layer(oln).output, output_layer_names))
    )
    return enc_model

#Or if we want them all, we can do like:
def generate_encapsulate_model(model):
    enc_model = Model(
        inputs=model.input,
        # ignore 1st layer (input), since some old models do not have 1st layer as Keras layer
        outputs=list(map(lambda layer: layer.output, model.layers[1:]))
    )
    return enc_model

In [16]:
enc_model = generate_encapsulate_model_with_output_layer_names(model, output_layer_names)
# OR
# enc_model = generate_encapsulate_model(model)

In [28]:
enc_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 24, 24, 6)         156       
_________________________________________________________________
MaxPooling2D_1 (MaxPooling2D (None, 12, 12, 6)         0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 8, 8, 16)          2416      
_________________________________________________________________
MaxPooling2D_2 (MaxPooling2D (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
__________

In [17]:
print(enc_model.predict(input_sample))

[array([[[[ 2.00877056e-01, -8.13024938e-02, -3.08210999e-02,
          -9.00693834e-02, -2.25125656e-01, -2.30446860e-01],
         [ 3.09659362e-01, -4.71497118e-01,  3.45869422e-01,
          -3.31146449e-01,  3.36842895e-01, -2.39532545e-01],
         [-8.89044255e-02, -1.73721522e-01, -3.86155993e-01,
          -3.93427074e-01,  4.42137599e-01, -4.58599627e-01],
         ...,
         [ 3.45173657e-01, -3.73328507e-01, -1.25077814e-01,
          -5.66822171e-01, -3.63146842e-01, -1.72495425e-01],
         [-2.37354681e-01, -4.48242307e-01,  4.44415733e-02,
          -9.97340232e-02,  1.11331567e-01, -1.21392608e-02],
         [ 4.50378954e-02, -2.70568788e-01,  3.91309798e-01,
          -1.79743752e-01,  2.43497893e-01, -3.95290181e-02]],

        [[ 1.64194599e-01, -2.46773481e-01, -2.06775814e-01,
          -4.67128009e-01,  1.10331312e-01, -2.18480200e-01],
         [ 4.36525166e-01, -1.56169638e-01,  3.92435431e-01,
          -4.49596733e-01,  4.14243698e-01, -4.06216830e-03],

3. Save encapsulated model

In [29]:
enc_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
enc_model.save("./models/enc_model.h5")
#save_model(enc_model, "./models/enc_model.h5")

4. Convert to TensorSpace compatible model

In [37]:
#type in terminal instead
tensorflowjs_converter --input_format=keras ./models/enc_model.h5 ./models/json_models/keras

SyntaxError: invalid syntax (<ipython-input-37-c488219b035e>, line 1)

In [44]:
enc_model.load( {

    type: "keras",
    url: './models/json_models/keras/model.json',
    multiInputs: true,
    inputShapes: [[100], [1]],

    onComplete: function() {

        console.log( "Complete load model." );

    }

} );

SyntaxError: invalid syntax (<ipython-input-44-bb1f2020ee4e>, line 8)